In [1]:
import os
import urllib, cStringIO

import pymongo as pm

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
from __future__ import division
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64
import json

### read in human annotations csv (prepped in pilot2_analysis_sketchpad_basic_recog.ipynb)

In [20]:
X = pd.read_csv('./sketchpad_basic_recog_group_data_2.csv')
print 'Shape of un-augmented (additional columns) annotation csv: {}'.format(X.shape)

X = pd.read_csv('./sketchpad_basic_recog_group_data_2_augmented.csv')
print 'Shape of augmented annotation csv: {}'.format(X.shape)

Shape of un-augmented (additional columns) annotation csv: (24976, 21)
Shape of augmented annotation csv: (24976, 23)


### prep similarity jsons for evaluation by passing forward to RSA

Format for json is a dictionary of dictionaries, where each top-level key refers to one of the renders, e.g. "cuckoo". For each render, you can look up the similarity with each sketch, referenced with an abbreviated ID taken by trimming the last 12-character string, and appending an underscore, and the trial number. E.g., 'gameID_9903-d6e6a9ff-a878-4bee-b2d5-26e2e239460a_trial_9.npy' ==> '26e2e239460a_9'

In [40]:
def load_json(json_path):
    with open(json_path) as fp:
        data = json.load(fp)  
    return data

def simplify_sketch(path): ## example path: 'gameID_9903-d6e6a9ff-a878-4bee-b2d5-26e2e239460a_trial_9.npy' ==> '26e2e239460a_9'
    path = '_'.join(os.path.splitext(os.path.basename(path))[0].split('_')[1:])
    path = path.split('-')[-1]
    path = path.replace('_trial', '')
    return path

def add_simplified_ids(X):
    ## add renderID and sketchID to dataframen
    sketchID = []
    for i,d in X.iterrows():
        sketchID.append(simplify_sketch(d['fname']))
    X['sketchID'] = sketchID    
    return X


from collections import Counter
import analysis_helpers as h
reload(h)
## get standardized object list
categories = ['bird','car','chair','dog']
obj_list = []
for cat in categories:
    for i,j in h.objcat.iteritems():
        if j==cat:
            obj_list.append(i)  

In [22]:
## add sketchid to the dataframe
X = add_simplified_ids(X)

In [ ]:
## compute similarities for each sketch based on annotations

In [34]:
sketch_list = np.unique(X['fname'])

In [43]:
this_sketch = sketch_list[0]
prob = np.zeros(len(obj_list)) ## initialize the probability vector
choices = X[X['fname']==this_sketch]['choice'].values ## get list of all choices

In [48]:
## get counter dictionary
cdict = Counter(choices)
## populate probability vector accordingly
for i,o in enumerate(obj_list):
    if o in cdict:
        prob[i] = cdict[o]

In [58]:
prob/np.sum(prob)

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.11111111,
       0.33333333, 0.        , 0.        , 0.        , 0.        ,
       0.33333333, 0.22222222])

### generate cost dictionaries

### generate condition-lookup.json to be able to pair sketches with condition